In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [5]:
df = pd.read_csv("retail_store_inventory.csv")
df.head(2)

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                73100 non-null  object 
 1   Store ID            73100 non-null  object 
 2   Product ID          73100 non-null  object 
 3   Category            73100 non-null  object 
 4   Region              73100 non-null  object 
 5   Inventory Level     73100 non-null  int64  
 6   Units Sold          73100 non-null  int64  
 7   Units Ordered       73100 non-null  int64  
 8   Demand Forecast     73100 non-null  float64
 9   Price               73100 non-null  float64
 10  Discount            73100 non-null  int64  
 11  Weather Condition   73100 non-null  object 
 12  Holiday/Promotion   73100 non-null  int64  
 13  Competitor Pricing  73100 non-null  float64
 14  Seasonality         73100 non-null  object 
dtypes: float64(3), int64(5), object(7)
memory usage: 8.4+

In [8]:
# Convert date object -> datetime
df["Date"] = pd.to_datetime(df["Date"], errors = "coerce")
df = df.sort_values(["Date"] + ["Store ID","Product ID"]).reset_index(drop = True)

# Coerce non-numeric values
for c in ["Inventory Level", "Units Sold", "Units Ordered", "Demand Forecast", "Price", "Discount", "Competitor Pricing"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors = "coerce")

In [10]:
df.shape

(73100, 15)

In [12]:
df.isna().sum()

Date                  0
Store ID              0
Product ID            0
Category              0
Region                0
Inventory Level       0
Units Sold            0
Units Ordered         0
Demand Forecast       0
Price                 0
Discount              0
Weather Condition     0
Holiday/Promotion     0
Competitor Pricing    0
Seasonality           0
dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                73100 non-null  datetime64[ns]
 1   Store ID            73100 non-null  object        
 2   Product ID          73100 non-null  object        
 3   Category            73100 non-null  object        
 4   Region              73100 non-null  object        
 5   Inventory Level     73100 non-null  int64         
 6   Units Sold          73100 non-null  int64         
 7   Units Ordered       73100 non-null  int64         
 8   Demand Forecast     73100 non-null  float64       
 9   Price               73100 non-null  float64       
 10  Discount            73100 non-null  int64         
 11  Weather Condition   73100 non-null  object        
 12  Holiday/Promotion   73100 non-null  int64         
 13  Competitor Pricing  73100 non-null  float64   

In [15]:
df.nunique().sort_values()

Holiday/Promotion         2
Region                    4
Weather Condition         4
Seasonality               4
Discount                  5
Store ID                  5
Category                  5
Product ID               20
Units Ordered           181
Inventory Level         451
Units Sold              498
Date                    731
Price                  8999
Competitor Pricing     9751
Demand Forecast       31608
dtype: int64

In [18]:
df["Date"].min(), df["Date"].max()

(Timestamp('2022-01-01 00:00:00'), Timestamp('2024-01-01 00:00:00'))

In [19]:
# Feature Engineering
df = df.sort_values([*["Store ID", "Product ID"], "Date"]).reset_index(drop = True)
df["dow"] = df["Date"].dt.dayofweek       # 0 = Monday
df["week"] = df["Date"].dt.isocalendar().week.astype(int)
df["month"] = df["Date"].dt.month
df["quarter"] = df["Date"].dt.quarter
df["year"] = df["Date"].dt.year
df["is_month_start"] = df["Date"].dt.is_month_start.astype(int)
df["is_month_end"] = df["Date"].dt.is_month_end.astype(int)

In [21]:
df["discount_pct"] = df["Discount"] / 100.0
df["net_price"] = df["Price"] * (1 - df["discount_pct"])
df["competitor_gap"] = df["net_price"] - df["Competitor Pricing"]

In [23]:
df

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,...,dow,week,month,quarter,year,is_month_start,is_month_end,discount_pct,net_price,competitor_gap
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,...,5,52,1,1,2022,1,0,0.20,26.8000,-2.8900
1,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,...,6,52,1,1,2022,0,0,0.10,25.1550,-5.7350
2,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,...,0,1,1,1,2022,0,0,0.20,50.1600,-8.0600
3,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,...,1,1,1,1,2022,0,0,0.15,66.1980,-9.7920
4,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,...,2,1,1,1,2022,0,0,0.20,22.7680,-6.6320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73095,2023-12-28,S005,P0020,Groceries,South,198,56,27,50.18,21.75,...,3,52,12,4,2023,0,0,0.05,20.6625,-4.6275
73096,2023-12-29,S005,P0020,Clothing,East,446,268,30,267.54,85.58,...,4,52,12,4,2023,0,0,0.20,68.4640,-19.1660
73097,2023-12-30,S005,P0020,Toys,North,251,149,181,162.92,79.48,...,5,52,12,4,2023,0,0,0.10,71.5320,-11.1580
73098,2023-12-31,S005,P0020,Furniture,East,64,40,99,59.69,90.79,...,6,52,12,4,2023,0,1,0.05,86.2505,-5.4195


In [24]:
# Create lags & rolling windows for time-series analyses

LAGS = [1, 7, 14, 28]
ROLL_WINDOWS = [7, 28]

def add_group_lags_rolls(
    frame: pd.DataFrame,
    lags = LAGS,
    rolls = ROLL_WINDOWS,
    group_cols = ("Store ID", "Product ID"),
    target = "Units Sold",
    date_col = "Date"):
    
    # Ensure proper ordering within each group
    out = frame.sort_values(list(group_cols) + [date_col]).copy()

    # Group once; keep sort=False to preserve row order after sort_values
    g = out.groupby(list(group_cols), sort=False)

    for l in lags:
        out[f"{target}_lag{l}"] = g[target].shift(l)

    # Rolling means over strictly past values (shift by 1 first)
    for w in rolls:
        minp = max(2, w // 2)  # avoid too-narrow windows
        out[f"{target}_rmean{w}"] = g[target].transform(
            lambda s: s.shift(1).rolling(window=w, min_periods=minp).mean()
        )

    return out
    
df = add_group_lags_rolls(df)

In [26]:
df.head(20)

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,...,is_month_end,discount_pct,net_price,competitor_gap,Units Sold_lag1,Units Sold_lag7,Units Sold_lag14,Units Sold_lag28,Units Sold_rmean7,Units Sold_rmean28
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,...,0,0.20,26.8000,-2.8900,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,...,0,0.10,25.1550,-5.7350,127.0,NaN,NaN,NaN,NaN,NaN
2,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,...,0,0.20,50.1600,-8.0600,81.0,NaN,NaN,NaN,NaN,NaN
3,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,...,0,0.15,66.1980,-9.7920,5.0,NaN,NaN,NaN,71.000000,NaN
4,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,...,0,0.20,22.7680,-6.6320,58.0,NaN,NaN,NaN,67.750000,NaN
5,2022-01-06,S001,P0001,Electronics,East,198,37,155,39.09,91.05,...,0,0.05,86.4975,-0.8025,147.0,NaN,NaN,NaN,83.600000,NaN
6,2022-01-07,S001,P0001,Furniture,North,195,107,106,117.92,53.99,...,0,0.10,48.5910,-10.2090,37.0,NaN,NaN,NaN,75.833333,NaN
7,2022-01-08,S001,P0001,Furniture,East,231,2,119,0.84,66.30,...,0,0.20,53.0400,-10.7800,107.0,127.0,NaN,NaN,80.285714,NaN
8,2022-01-09,S001,P0001,Electronics,South,373,350,178,352.24,41.72,...,0,0.00,41.7200,3.0400,2.0,81.0,NaN,NaN,62.428571,NaN
9,2022-01-10,S001,P0001,Electronics,West,327,36,132,39.50,32.68,...,0,0.05,31.0460,-3.6640,350.0,5.0,NaN,NaN,100.857143,NaN


In [27]:
df.to_csv("modified.csv", index = False)

In [ ]:
# TODO: Drop/impute NaNs created from lags & moving averages
#       One-hot encode categorical variables
#       Scale numerical values
#       Train/Test/Validation Split